In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
df = pd.read_csv('/Users/lokikeeler/Downloads/train-balanced-sarcasm_2.csv')

In [4]:
df.head()

,label,comment,score,ups,downs,date,created_utc,parent_comment,year,SUB_2007scape,...,SUB_television,SUB_tf2,SUB_todayilearned,SUB_trees,SUB_ukpolitics,SUB_unitedkingdom,SUB_videos,SUB_worldnews,SUB_wow,SUB_xboxone
0,0,NC and NH.,2,-1,-1,2016-10-01,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ...",2016,False,...,False,False,False,False,False,False,False,False,False,False
1,0,You do know west teams play against west teams...,-4,-1,-1,2016-11-01,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...,2016,False,...,False,False,False,False,False,False,False,False,False,False
2,0,"They were underdogs earlier today, but since G...",3,3,0,2016-09-01,2016-09-22 21:45:37,They're favored to win.,2016,False,...,False,False,False,False,False,False,False,False,False,False
3,0,"This meme isn't funny none of the ""new york ni...",-8,-1,-1,2016-10-01,2016-10-18 21:03:47,deadass don't kill my buzz,2016,False,...,False,False,False,False,False,False,False,False,False,False
4,0,"I don't pay attention to her, but as long as s...",0,0,0,2016-09-01,2016-09-02 10:35:08,do you find ariana grande sexy ?,2016,False,...,False,False,False,False,False,False,False,False,False,False


In [5]:
df = df.drop(columns='parent_comment')

In [8]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

ENGLISH_STOP_WORDS = stopwords.words('english')

def my_tokenizer(sentence):
    stemmer = nltk.stem.PorterStemmer()
    # remove punctuation and set to lower case
    for punctuation_mark in string.punctuation:
        sentence = sentence.replace(punctuation_mark,'').lower()
        #took out lower case converter

    # split sentence into words
    listofwords = sentence.split(' ')
    listofstemmed_words = []
    
    # remove stopwords and any tokens that are just empty strings
    for word in listofwords:
        if (not word in ENGLISH_STOP_WORDS) and (word!=''):
            # Stem words
            stemmed_word = stemmer.stem(word)
            listofstemmed_words.append(stemmed_word)

    return listofstemmed_words  


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lokikeeler/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lokikeeler/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
X = df.drop(columns='label')
y = df['label']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
X_train.shape

(384932, 107)

In [13]:
# using our custom tokenizer in TfidfVectorizer

tfidf = TfidfVectorizer(
                        # words must appear at least 5 times
                        min_df=5,
                        # Stemming, lemmatization, stop word removal, tokenization
                        tokenizer=my_tokenizer, 
                        #ngrams included
                        ngram_range=(1, 3)
                        )
tfidf.fit(X_train['comment'])

X_train_transformed = tfidf.transform(X_train['comment'])
X_test_transformed = tfidf.transform(X_test['comment'])

X_train_transformed.shape

/Users/lokikeeler/anaconda3/envs/capstone/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


(384932, 63771)

In [18]:
np.random.seed(123)

# Fitting a model
logreg = LogisticRegression(C=0.1)
logreg.fit(X_train_transformed, y_train)

# Training and test score
print(f"Train score: {logreg.score(X_train_transformed, y_train)}")
print(f"Test score: {logreg.score(X_test_transformed, y_test)}")

Train score: 0.6988299232072158
Test score: 0.6823703416337318


In [18]:
def sarcasm_detector(sentence):   
    vector1 = TfidfVectorizer().transform(sentence)
    prediction = logreg.predict(vector1)
    return prediction

In [20]:
tfidf_2 = TfidfVectorizer(
                        tokenizer=my_tokenizer, 
                        )

In [21]:
def sarcasm_detector_2(sentence):   
    vector1 = tfidf_2.transform(sentence)
    prediction = logreg.predict(vector1)
    return prediction

In [22]:
sarcasm_detector("Wow, great lesson today, bud")

NotFittedError: The TF-IDF vectorizer is not fitted

In [19]:
# Instantiate & fit the DT
DT_model = DecisionTreeClassifier(max_depth=10)
DT_model.fit(X_train_transformed, y_train)

# Evaluate its classification accuracy (Just on the training set for now)
print(f"DT training set accuracy: {DT_model.score(X_train_transformed, y_train)}")
print(f"DT test set accuracy: {DT_model.score(X_test_transformed, y_test)}")

DT training set accuracy: 0.9234436212110191
DT test set accuracy: 0.6351320224038018


In [ ]:
clf = DecisionTreeClassifier(
    max_depth=20,
    min_samples_split=0.05,  # 5% of total data size
    min_samples_leaf=0.02,   # 2% of total data size
    max_leaf_nodes=100,
    max_features='sqrt',
    min_impurity_decrease=0.001
)

In [ ]:
# decission tree
# logisitic
# IDF

In [ ]:
# took out lower case remover (so it allows it)
# TF IDF
# CONFUSION MATRICES

In [ ]:
# NEXT SPRINT
# improve accuracy of decision tree
# improve with random forrest
